
\begin{titlepage}
\centering
{\bfseries\LARGE Universidad Nacional San Agustin de Arequipa \par}
\vspace{1cm}
{\scshape\Large Facultad de Ingenierias de Produccion y Servicios \par}
\vspace{3cm}
{\scshape\Huge Escuela Profesional de Ingenieria de Sistemas \par}
\vspace{3cm}
{\itshape\Large Inteligencia Artificial \par}
\vfill
{\Large Alumno: \par}
{\Large Fuentes Paredes Nelson Alejandro \par}
\vfill
{\Large Mayo 2020 \par}
\end{titlepage}

# Importado Librerias

In [1]:
import pprint
import math
import pandas as pd

# Metodo para calcular la Entriopia

In [2]:
def entropy(data):
    size = len(data)
    positive = len(data.loc[data['Clase'] == '+'])
    negative = len(data.loc[data['Clase'] == '-'])
    if positive > 0:
        positive = (positive / size) * math.log2(positive / size)
    else:
        positive = 0
    if negative > 0:
        negative = negative / size * math.log2(negative / size)
    else:
        negative = 0
    return - positive - negative

# Metodo Para calcular la ganancia de un atributo
Este metodo también retorna una diccionario con la entriopia de cada valor posible del atributo,recibe la data y las columnas sobre laa que tiene que trabajar y la entropia del atributo anterior.

In [3]:
def earning(data, column, entropy_current):
    earn = entropy_current
    unique_values = data[column].unique()
    entropies = {}
    for value in unique_values:
        data_value = data.loc[data[column] == value]
        entropy_value = entropy(data_value)
        entropies[value] = entropy_value
        earn = earn - len(data_value) / len(data) * entropy_value
    return earn, entropies

# Metodo para la contrucción del árbol y nodo
Es un metodo recursivo que recibe los datos y columnas que van quedando y aplica el algoritmo ID3

In [4]:
def build_node(data, columns):
    entropy_data = entropy(data)
    dictionary = {}
    earnings = pd.DataFrame(columns=('Column', 'Earning'))
    entropies = {}
    
    # Obtención de la ganacia y entriopias de los valores del atributo
    for column in columns:
        earn_result = earning(data, column, entropy_data)
        earn = earn_result[0]
        entropies[column] = earn_result[1]
        earnings.loc[len(earnings)] = [column, earn]
    
    # Selección del atributo con más ganancia
    earnings = earnings.set_index('Column')
    major = earnings[['Earning']].idxmax().to_list()[0]
    
    # Creación del nuevo nodo con el valor resultado o un nuevo nodo recursivo
    node = entropies[major]
    columns = columns.drop(major)
    for value in node:
        new_data = data.loc[data[major] == value]
        if node[value] == 0:
            node[value] = new_data['Clase'].mode()[0]
        else:
            node[value] = build_node(new_data, columns)
    dictionary[major] = node
    return dictionary

# Metodo Principal
Recibe como parametro el nombre del archivo que va a usar

In [5]:
def main(file):
    data = pd.read_csv(file,  index_col=0)
    print(data)
    columns = data.columns.drop('Clase')
    return build_node(data, columns)

# Ejecución del Algoritmo

In [6]:
file = 'data.csv'
three = main(file)
print("\nÁRBOL RESULTADO\n")
pprint.pprint(three, width=1)

    Color     Forma   Tamaño Clase
Ej                                
1    rojo  cuadrado   grande     +
2    azul  cuadrado   grande     +
3    rojo   redondo  pequeño     -
4   verde  cuadrado  pequeño     -
5    rojo   redondo   grande     +
6   verde  cuadrado   grande     -

ÁRBOL RESULTADO

{'Color': {'azul': '+',
           'rojo': {'Tamaño': {'grande': '+',
                               'pequeño': '-'}},
           'verde': '-'}}
